## Set the prediction url for Custom Vision

In [ ]:
import requests
import json
import os
import numpy as np
import pandas as pd
import cv2 
import matplotlib.pyplot as plt
import copy
import time

url="https://japaneast.api.cognitive.microsoft.com/customvision/v3.0/Prediction/xxxxx/detect/iterations/xxxxx/image" # Prediction URL

headers={
        'content-type':'application/octet-stream',
        'Prediction-Key':'xxxxxxxxxxxxxxxxxxxxxxx' # Prediction key
        }

tag_list = ["tagA","tagB","tagC"] # Tag list

In [ ]:
def cv_oj_api(url, headers, img_file_path, img, tag_list):
    response = requests.post(url,data=open(img_file_path,"rb"),headers=headers)
    response.raise_for_status()
    result = response.json()
    tag_grab_dict = {}
    #print(result)
    prob = pd.DataFrame([[tag_i["tagName"],tag_i["probability"]] for tag_i in result["predictions"]],columns=["tagName","probability"])
    #print(prob.head)
    for tag in tag_list:
        if len(prob.query('tagName==@tag'))==0:
            tag_grab_dict[(tag+'_px')] = np.nan
            tag_grab_dict[(tag+'_py')] = np.nan
            tag_grab_dict[(tag+'_x')] = np.nan
            tag_grab_dict[(tag+'_y')] = np.nan
            continue
        if prob.query('tagName==@tag')['probability'].max() < 0.2:
            tag_grab_dict[(tag+'_px')] = np.nan
            tag_grab_dict[(tag+'_py')] = np.nan
            tag_grab_dict[(tag+'_x')] = np.nan
            tag_grab_dict[(tag+'_y')] = np.nan
            continue

        tag_ids = prob.query('tagName==@tag')['probability']        
        for i, probability in tag_ids.iteritems():
            if(probability > 0.5):
                tag_grid = result["predictions"][i]['boundingBox']
                y=int(tag_grid['top']*img.shape[0])
                x=int(tag_grid['left']*img.shape[1])
                h=int(tag_grid['height']*img.shape[0])
                w=int(tag_grid['width']*img.shape[1])
                #center of a box
                tag_x = x + int(w/2)
                tag_y = y + int(h/2)
                tag_grab_dict[(tag+'_px')] = x
                tag_grab_dict[(tag+'_py')] = y
                tag_grab_dict[(tag+'_x')] = tag_x
                tag_grab_dict[(tag+'_y')] = tag_y

                cv2.putText(img, tag, (x, y - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255) )

    return img, tag_grab_dict

In [ ]:
inpath = "./input/"
outpath = "./output/"
cap = cv2.VideoCapture(inpath+'org.mp4') # Video file name to be predicted
cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 10

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
out = cv2.VideoWriter((outpath +'result.mp4'), fourcc, fps, (cap_width, cap_height)) # Output file name

count=0
tag_grab_df = pd.DataFrame([])

while(cap.isOpened()):
    t1 = time.time()
    ret, frame = cap.read()
    if not ret:
        break
    count += 1

    # 1 frame/sec 
    if count % fps != 0:
        continue

    # img resize
    #height = frame.shape[0]
    #width = frame.shape[1]
    #frame = cv2.resize(frame, (int(width*0.5), int(height*0.5)))

    tmp_file_path = outpath + 'tmp.jpg'
    cv2.imwrite(tmp_file_path, frame)

    # api prediction & masking
    frame, tag_grab_dict = cv_oj_api(url, headers, tmp_file_path, frame, tag_list)
    
    tag_grab_dict['time'] = count/fps
    tag_grab_df = tag_grab_df.append([tag_grab_dict])

    # write & show frame
    #print(frame.shape)
    out.write(frame)
    img_file_path = outpath + str(count) + '.jpg'
    #cv2.imwrite(img_file_path, frame)
    #cv2_imshow(frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    sec = count/fps
    if sec % 10 == 0:
      print(sec,'sec ended.')

    #for short time debag
    #if sec >= 60:
    #     break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

tag_grab_df.to_csv(outpath+'tag_grab_df.csv', index=False, encoding='shift-JIS')

## Register an Azure blob container as a datastore

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

blob_datastore_name='xxxx' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "xxxx") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "xxxx") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "xxxx") # Storage account access key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [ ]:
# upload to blob
blob_datastore.upload(src_dir=outpath, target_path='results')